<a href="https://colab.research.google.com/github/Denver-Contabil/Colab/blob/master/Geradora_Extrato_XLS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Bibliotecas utilizadas

In [ ]:
!pip install xlwt
!pip install tabula-py
import pandas as pd
import tabula as tb
import re
import glob
import os
import xlwt

- Conversao dos PDF em CSV

In [ ]:
files = glob.glob('/content/drive/Shareddrives/Controladoria de Fundos/ID CTVM/Fundos/z. Extratos/Banco ID/*.pdf')

java_options = "-Dfile.encoding=UTF-8"

# Percorre cada arquivo PDF
for file in files:
    csv_filename = file.replace(".pdf", ".csv")

    tb.convert_into(file, csv_filename, output_format="csv", pages="all", java_options=java_options)

Leitura dos CSV e tratamento dos dados para exportacao em XLS

In [ ]:
csv_files = glob.glob('/content/drive/Shareddrives/Controladoria de Fundos/ID CTVM/Fundos/z. Extratos/Banco ID/*.csv')

for index in range(len(csv_files)):
  pdf = pd.read_csv(csv_files[index], decimal = ',')

  conta = pdf.iloc[1][1]

  conta = re.findall(r'\d+', conta)
  n_conta = f'{conta[1]}-{conta[2]}'
  pdf.iloc[:, 2] = pdf.iloc[:, 2].fillna(' ')

  pdf.iloc[:, 1] = pdf.iloc[:, 1] + ' ' + pdf.iloc[:, 2]

  pdf = pdf.drop(pdf.columns[2], axis=1)
  pdf = pdf.drop(pdf.columns[4], axis=1)

  pdf = pdf.drop(range(0, 8))
  pdf = pdf.dropna(axis = 0)

  def formatar_valor(valor):
      valor = str(valor)
      valor = valor.replace('(', '').replace(')', '').replace(' ', '').replace('+', '')
      if valor.endswith('-'):
          return '-' + valor[:-1]
      else:
          return valor

  pdf.iloc[:, 2] = pdf.iloc[:, 2].apply(formatar_valor)

  pdf['Numero Conta'] = n_conta

  pdf = pdf.rename(columns = {pdf.columns[0]: 'Data', pdf.columns[1]: 'Histórico', pdf.columns[2]: 'Valor', pdf.columns[3]: 'Saldo'})

  pdf_new = pdf[['Data', 'Numero Conta', 'Histórico', 'Valor', 'Saldo']]

  csv_files[index] = csv_files[index].replace('.csv', '')
  pdf_new.to_excel(f'{csv_files[index]}.xls', index = False)

  if os.path.exists(csv_files[index]):
    os.remove(csv_files[index]) # Esta linha esta em teste

<ipython-input-119-8fa5f3b60d4f>:37: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  pdf_new.to_excel(f'{csv_files[index]}_.xls', index = False)
<ipython-input-119-8fa5f3b60d4f>:37: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the 